In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
import gc
import time
import threading
from datetime import datetime
import urllib3
from itertools import permutations
import pandas as pd
import numpy as np
from numpy import vectorize
from numpy.random import SeedSequence
from types import FunctionType
from collections import namedtuple

EXECUTION_NAME = datetime.now().replace(second=0,microsecond=0).isoformat()
TRACE_NAME = 
CHECKPOINT_PATH = '../output/';
EXECUTION_DURATION_S = 14*24*60*60
NS_TO_S = 1000000000
max_int = 1000000000; 
min_int = -1000000000; 
num_base = 10;

def string_to_file(filename, content):
    with open(CHECKPOINT_PATH+EXECUTION_NAME+f'-{filename}.csv','w') as f:
        f.write(str(content)[1:-1])
        
seedz = get_true_random(n_samples, False)
print('random seeds: ',seedz)
string_to_file('SEED',seedz)
RANDOM_GEN = np.random.default_rng(seed=SeedSequence(entropy=list(map(abs,seedz))))

random seeds:  [236756603, 111787606, -434135063, -367674061, 213947742, -160780412, 680182866, 697314932, -662565324, 994103395, 403220169, 487317545, 449218264, 545009335, 10677032, -605960024, -813662505, -409913948, -402388281, -867505113, -302362408, 67082660, 958339104, -231325652, 663778762, 191104139, -208828714, 940991258, 747580595, -544828770, 441053003, 607118037, -984716308, 385750719, -765762784, 847862591, 954183295, -773026378, 797396895, -522773503, 37117581, -653914968, 497833991, 358987007, 448996891, -332305826, 94902095, -651496514, 13438596, -45196826, 167115983, -697239988, 93592755, 456641666, 97061316, 212621856, 897108520, 666972363, 354041323, 16842204, 480126838, -920056466, -733690652, 578136639, -990286244, -654425462, -503114496, 623777122, 390245934, 744505964, 331614125, -570437389, -298135554, 523183400, -291899579, -104974752, 385042331, -264273845, 41875769, -727821695, 554592548, 273551252, 538157670, 419268116, -209028719, -293861394, -472922581, -

In [2]:
_df : (pd.DataFrame | None) = pd.read_csv("../raw/AzureFunctionsInvocationTraceForTwoWeeksJan2021.tar.xz")
_df

,app,func,end_timestamp,duration
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,7.949090e-02,0.078
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,5.715786e+01,57.154
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,5.913048e+01,59.125
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,6.252541e+00,6.236
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,6.682396e-02,0.050
...,...,...,...,...
1980946,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209597e+06,0.001
1980947,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209598e+06,0.001
1980948,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001
1980949,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001


In [3]:
app_idx = _df["app"].unique()
RANDOM_GEN.shuffle(app_idx)
app_idx_enum = dict([x, idx] for idx, x
                    in enumerate(app_idx))
app_id_map = lambda app_id: app_idx_enum[app_id]
_df["app_id"] = _df["app"].apply(func=app_id_map)
func_idx = _df["func"].unique()
RANDOM_GEN.shuffle(func_idx)
func_idx_enum = dict([x, idx] for idx, x
                    in enumerate(func_idx))
func_id_map = lambda func_idx: func_idx_enum[func_idx]
_df["func_id"] = _df["func"].apply(func=func_id_map)
_df["end_ns"] = (NS_TO_S * _df["end_timestamp"]).astype(int)
_df["dur_ns"] = (NS_TO_S * _df["duration"]).astype(int)
_df['beg_ns'] = _df['end_ns'] - _df["dur_ns"]
_df["ts"] = _df['beg_ns']
_df.sort_values(by=["ts"],ignore_index=True, ascending=True, inplace=True)
_df["req_id"] = _df.index
_df["event_type"] = 1
_df

,app,func,end_timestamp,duration,app_id,func_id,end_ns,dur_ns,beg_ns,ts,req_id,event_type
0,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,e3cdb48830f66eb8689cc0223514569a69812b77e6611e...,7.949090e-02,0.078,115,146,79490900,78000000,1490900,1490900,0,1
1,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,337cd24a7d5fd5c92460faee4ebe6a186a0eb322bd17b7...,5.715786e+01,57.154,24,194,57157860040,57154000000,3860040,3860040,1,1
2,1573b95c039e51cc012b543a4af3bc7c3ee9485acbb003...,48cc770d590d3c5a7691b3b4e9302f82ec3be5ddc2a037...,5.913048e+01,59.125,24,372,59130477905,59125000000,5477905,5477905,2,1
3,f274d71de386ccc77e4ca74766dbc485461c3053059d47...,3d2aee54a133509f16fb636d74128c2adcfcac71c6dcef...,6.252541e+00,6.236,102,344,6252541065,6236000000,16541065,16541065,3,1
4,7b2c43a2bc30f6bb438074df88b603d2cb982d3e7961de...,68bbfd828223a505d7917339f4656c5f33ff93225cdb9d...,6.682396e-02,0.050,115,228,66823959,50000000,16823959,16823959,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1980946,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209597e+06,0.001,82,49,1209596701016903,1000000,1209596700016903,1209596700016903,1980946,1
1980947,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209598e+06,0.001,82,49,1209598191409111,1000000,1209598190409111,1209598190409111,1980947,1
1980948,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001,82,49,1209598561518907,1000000,1209598560518907,1209598560518907,1980948,1
1980949,a594f92f84072b4cd031fe5283d1781a6e98f430696dec...,155e47f8e7f751d0c845049456d01832013c61336a8cd8...,1.209599e+06,0.001,82,49,1209598628381967,1000000,1209598627381967,1209598627381967,1980949,1


In [4]:
_df.drop(columns=["app","func","end_timestamp","duration"], inplace=True)
_df.to_csv(CHECKPOINT_PATH+EXECUTION_NAME+"-PREPROCESS.csv.gz",
           columns=["ts","dur_ns","func_id","app_id"], index=True,
           index_label="id")

In [5]:
_df

,app_id,func_id,end_ns,dur_ns,beg_ns,ts,req_id,event_type
0,115,146,79490900,78000000,1490900,1490900,0,1
1,24,194,57157860040,57154000000,3860040,3860040,1,1
2,24,372,59130477905,59125000000,5477905,5477905,2,1
3,102,344,6252541065,6236000000,16541065,16541065,3,1
4,115,228,66823959,50000000,16823959,16823959,4,1
...,...,...,...,...,...,...,...,...
1980946,82,49,1209596701016903,1000000,1209596700016903,1209596700016903,1980946,1
1980947,82,49,1209598191409111,1000000,1209598190409111,1209598190409111,1980947,1
1980948,82,49,1209598561518907,1000000,1209598560518907,1209598560518907,1980948,1
1980949,82,49,1209598628381967,1000000,1209598627381967,1209598627381967,1980949,1


In [6]:
_df_dept = _df.sort_values(by="end_ns", ignore_index=True, ascending=True)
_df_dept["ts"] = _df["end_ns"]
_df_dept["event_type"] = 0
_df_all = pd.concat([_df, _df_dept])
_df_all.sort_values(by=["ts","event_type"],ignore_index = True, ascending=[True,False], inplace = True)

In [7]:
DF = (_df_all.copy(deep=True))[["ts", "event_type", "req_id", "func_id", "app_id", "beg_ns", "end_ns", "dur_ns" ]]
del _df, _df_dept, _df_all
gc.collect()

0

In [8]:
DF.to_csv(CHECKPOINT_PATH+EXECUTION_NAME+'-EVENTS.csv.gz',index=True, index_label="id")
DF  # this is a checkpoint for event preprocessing, next we'll augment data with some metrics

,ts,event_type,req_id,func_id,app_id,beg_ns,end_ns,dur_ns
0,1490900,1,0,146,115,1490900,79490900,78000000
1,3860040,1,1,194,24,3860040,57157860040,57154000000
2,5477905,1,2,372,24,5477905,59130477905,59125000000
3,16541065,1,3,344,102,16541065,6252541065,6236000000
4,16823959,1,4,228,115,16823959,66823959,50000000
...,...,...,...,...,...,...,...,...
3961897,1209598561518907,0,1980948,49,82,1209598560518907,1209598561518907,1000000
3961898,1209598627381967,1,1980949,49,82,1209598627381967,1209598628381967,1000000
3961899,1209598628381967,0,1980949,49,82,1209598627381967,1209598628381967,1000000
3961900,1209599693463968,1,1980950,49,82,1209599693463968,1209599694463968,1000000


In [9]:
print("DONE")

DONE
